In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix, precision_recall_curve
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
# Đọc dữ liệu
df = pd.read_csv('churn_customer.csv')

# Xử lý định dạng cho các cột số có dấu phẩy
df['balance'] = df['balance'].str.replace(',', '.').astype(float)
df['estimated_salary'] = df['estimated_salary'].str.replace(',', '.').astype(float)

In [28]:
# 1. Phân tích mức độ mất cân bằng dữ liệu
print("Phân bố lớp trong dữ liệu gốc:")
print(df['churn'].value_counts(normalize=True))

# Vẽ biểu đồ phân bố lớp
plt.figure(figsize=(6, 4))
sns.countplot(x='churn', data=df)
plt.title('Phân bố lớp Churn')
plt.savefig('class_distribution.png')
plt.close()

Phân bố lớp trong dữ liệu gốc:
churn
0    0.7963
1    0.2037
Name: proportion, dtype: float64


In [29]:
# 2. Xử lý dữ liệu
# Loại bỏ cột không cần thiết
df = df.drop('customer_id', axis=1)

# Mã hóa biến phân loại
le = LabelEncoder()
df['gender'] = le.fit_transform(df['gender'])
df = pd.get_dummies(df, columns=['country'], drop_first=True)

# Chuẩn hóa biến số
scaler = StandardScaler()
numeric_cols = ['credit_score', 'age', 'tenure', 'balance', 'products_number', 'estimated_salary']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Chia tập train/test
X = df.drop('churn', axis=1)
y = df['churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



In [30]:
# 3. Xử lý mất cân bằng dữ liệu
sampling_methods = {
    'SMOTE': SMOTE(random_state=42),
    'RUS': RandomUnderSampler(random_state=42),
    'ADASYN': ADASYN(random_state=42),
    'SMOTEENN': SMOTEENN(random_state=42)
}

train_data = {}
for name, sampler in sampling_methods.items():
    X_train_s, y_train_s = sampler.fit_resample(X_train, y_train)
    train_data[name] = (X_train_s, y_train_s)
    print(f"\nPhân bố lớp sau khi áp dụng {name}:")
    print(pd.Series(y_train_s).value_counts(normalize=True))


Phân bố lớp sau khi áp dụng SMOTE:
churn
1    0.5
0    0.5
Name: proportion, dtype: float64

Phân bố lớp sau khi áp dụng RUS:
churn
0    0.5
1    0.5
Name: proportion, dtype: float64

Phân bố lớp sau khi áp dụng ADASYN:
churn
0    0.511482
1    0.488518
Name: proportion, dtype: float64

Phân bố lớp sau khi áp dụng SMOTEENN:
churn
1    0.574593
0    0.425407
Name: proportion, dtype: float64


In [32]:
# 4. Định nghĩa các mô hình và lưới siêu tham số
models = {
    'Logistic Regression': {
        'model': LogisticRegression(max_iter=1000, random_state=42),
        'param_grid': {
            'C': [0.001, 0.01, 0.1, 1, 10, 100],
            'solver': ['liblinear', 'lbfgs', 'saga'],
            'class_weight': ['balanced', None]
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(random_state=42),
        'param_grid': {
            'n_estimators': [100, 200],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(random_state=42, eval_metric='logloss'),
        'param_grid': {
            'n_estimators': [100, 200],
            'max_depth': [3, 6, 9],
            'learning_rate': [0.01, 0.1, 0.3]
        }
    }
}

In [33]:
# 5. Hàm đánh giá với ngưỡng
def evaluate_with_threshold(model, X_train, y_train, X_test, y_test, threshold=0.5):
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred_adjusted = (y_pred_proba >= threshold).astype(int)

    print(f"\nĐánh giá mô hình {model.__class__.__name__} với ngưỡng {threshold}:")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred_adjusted))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_adjusted))

    return recall_score(y_test, y_pred_adjusted, pos_label=1)
    # 6. Thử nghiệm và tối ưu hóa các mô hình
results = {}
best_models = {}
best_recall = 0
best_model_name = ""
best_threshold = 0.5

for name, config in models.items():
    print(f"\n=== Đánh giá mô hình {name} ===")

    for sampling_name, (X_train_s, y_train_s) in train_data.items():
        print(f"\nSử dụng {sampling_name}:")
        grid_search = GridSearchCV(config['model'], config['param_grid'], cv=5, scoring='recall', n_jobs=-1)
        grid_search.fit(X_train_s, y_train_s)
        best_model = grid_search.best_estimator_
        best_models[f"{name} ({sampling_name})"] = best_model

        # Thử nghiệm với các ngưỡng thấp hơn để tối ưu Recall
        thresholds = [0.2, 0.25, 0.3, 0.4, 0.5]
        best_recall_model = 0
        best_threshold_model = 0.5
        for threshold in thresholds:
            recall = evaluate_with_threshold(best_model, X_train_s, y_train_s, X_test, y_test, threshold)
            if recall > best_recall_model:
                best_recall_model = recall
                best_threshold_model = threshold

        results[f"{name} ({sampling_name})"] = {
            'Recall (Class 1)': best_recall_model,
            'Best Threshold': best_threshold_model,
            'Best Params': grid_search.best_params_
        }

        if best_recall_model > best_recall:
            best_recall = best_recall_model
            best_model_name = f"{name} ({sampling_name})"
            best_threshold = best_threshold_model


=== Đánh giá mô hình Logistic Regression ===

Sử dụng SMOTE:

Đánh giá mô hình LogisticRegression với ngưỡng 0.2:
Confusion Matrix:
[[ 376 1217]
 [  17  390]]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.24      0.38      1593
           1       0.24      0.96      0.39       407

    accuracy                           0.38      2000
   macro avg       0.60      0.60      0.38      2000
weighted avg       0.81      0.38      0.38      2000


Đánh giá mô hình LogisticRegression với ngưỡng 0.25:
Confusion Matrix:
[[ 535 1058]
 [  32  375]]

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.34      0.50      1593
           1       0.26      0.92      0.41       407

    accuracy                           0.46      2000
   macro avg       0.60      0.63      0.45      2000
weighted avg       0.80      0.46      0.48      2000


Đánh giá mô hình LogisticRegression vớ

In [34]:
# 7. Phân tích tầm quan trọng đặc trưng
# Logistic Regression
if 'Logistic Regression' in best_model_name:
    coef = best_models[best_model_name].coef_[0]
    feature_importance = pd.DataFrame({'Feature': X.columns, 'Coefficient': coef})
    print("\nTầm quan trọng của các đặc trưng trong Logistic Regression:")
    print(feature_importance.sort_values(by='Coefficient', ascending=False))
    feature_importance.plot(kind='bar', x='Feature', y='Coefficient', figsize=(10, 6))
    plt.title('Tầm quan trọng đặc trưng - Logistic Regression')
    plt.savefig('feature_importance_logistic.png')
    plt.close()

# Random Forest
if 'Random Forest' in best_model_name:
    importances = best_models[best_model_name].feature_importances_
    feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
    print("\nTầm quan trọng của các đặc trưng trong Random Forest:")
    print(feature_importance.sort_values(by='Importance', ascending=False))
    feature_importance.plot(kind='bar', x='Feature', y='Importance', figsize=(10, 6))
    plt.title('Tầm quan trọng đặc trưng - Random Forest')
    plt.savefig('feature_importance_rf.png')
    plt.close()


Tầm quan trọng của các đặc trưng trong Logistic Regression:
             Feature  Coefficient
2                age     0.651297
9    country_Germany     0.385673
4            balance     0.259302
8   estimated_salary     0.040853
3             tenure    -0.004982
10     country_Spain    -0.011238
0       credit_score    -0.046877
6        credit_card    -0.080066
5    products_number    -0.143842
1             gender    -0.311978
7      active_member    -0.414062


In [35]:
# 8. Vẽ Precision-Recall Curve cho mô hình tốt nhất
y_pred_proba = best_models[best_model_name].predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'Precision-Recall Curve ({best_model_name})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall Curve for {best_model_name}')
plt.legend()
plt.savefig('precision_recall_curve.png')
plt.close()

In [36]:
# 9. In kết quả
print("\nKết quả các mô hình:")
print(pd.DataFrame(results).T)


Kết quả các mô hình:
                               Recall (Class 1) Best Threshold  \
Logistic Regression (SMOTE)            0.958231            0.2   
Logistic Regression (RUS)               0.97543            0.2   
Logistic Regression (ADASYN)           0.965602            0.2   
Logistic Regression (SMOTEENN)         0.995086            0.2   
Random Forest (SMOTE)                  0.862408            0.2   
Random Forest (RUS)                    0.972973            0.2   
Random Forest (ADASYN)                 0.891892            0.2   
Random Forest (SMOTEENN)               0.889435            0.2   
XGBoost (SMOTE)                        0.660934            0.2   
XGBoost (RUS)                          0.963145            0.2   
XGBoost (ADASYN)                       0.636364            0.2   
XGBoost (SMOTEENN)                     0.791155            0.2   

                                                                      Best Params  
Logistic Regression (SMOTE)     {'C

In [37]:
# 10. Mô hình tốt nhất
print(f"\nMô hình tốt nhất: {best_model_name}")
print(f"Recall lớp 1: {best_recall}")
print(f"Ngưỡng tốt nhất: {best_threshold}")
print(f"Kết quả chi tiết: {results[best_model_name]}")

# 11. Lý do chọn mô hình
print("\nLý do chọn mô hình:")
if 'Logistic Regression' in best_model_name:
    print("Logistic Regression được chọn vì đạt Recall lớp 1 cao, đồng thời có tính đơn giản, dễ diễn giải, và phù hợp với bài toán cần giải thích các yếu tố ảnh hưởng đến churn.")
else:
    print(f"{best_model_name} được chọn vì đạt Recall lớp 1 cao nhất ({best_recall}). Tuy nhiên, Logistic Regression (RUS) với Recall 0.9042 là lựa chọn thay thế tốt nhờ tính đơn giản và khả năng diễn giải, phù hợp với các ứng dụng cần giải thích rõ ràng.")


Mô hình tốt nhất: Logistic Regression (SMOTEENN)
Recall lớp 1: 0.995085995085995
Ngưỡng tốt nhất: 0.2
Kết quả chi tiết: {'Recall (Class 1)': 0.995085995085995, 'Best Threshold': 0.2, 'Best Params': {'C': 0.001, 'class_weight': None, 'solver': 'lbfgs'}}

Lý do chọn mô hình:
Logistic Regression được chọn vì đạt Recall lớp 1 cao, đồng thời có tính đơn giản, dễ diễn giải, và phù hợp với bài toán cần giải thích các yếu tố ảnh hưởng đến churn.
